# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, observables=[])

In [3]:
# as dict
dict(vocs)

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'observables': []}

In [4]:
#  re-parse dict
vocs2 = VOCS.from_dict(dict(vocs))

In [5]:
# Check that these are the same
vocs2 == vocs

True

In [6]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [7]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [8]:
# json
vocs.to_json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"observables":[]}'

# Objective Evaluation

In [9]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,448.121438,-0.935596,1234,447.185842,449.057034,894.371683,898.114068
6,772.541478,-0.877792,1234,771.663686,773.419270,1543.327372,1546.838540
7,789.409249,0.985747,1234,790.394996,788.423502,1580.789992,1576.847004
8,833.926167,-0.352108,1234,833.574060,834.278275,1667.148119,1668.556550
9,496.442917,0.682281,1234,497.125198,495.760636,994.250396,991.521271
10,217.430395,0.554917,1234,217.985312,216.875478,435.970623,433.750956
11,770.163380,-0.871021,1234,769.292359,771.034400,1538.584717,1542.068801
12,614.842277,0.201315,1234,615.043592,614.640962,1230.087184,1229.281925
13,834.535411,0.282842,1234,834.818253,834.252569,1669.636505,1668.505138
14,443.285500,-0.875661,1234,442.409839,444.161161,884.819679,888.322323


In [10]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [11]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-447.185842,449.057034
6,-771.663686,773.419270
7,-790.394996,788.423502
8,-833.574060,834.278275
9,-497.125198,495.760636
10,-217.985312,216.875478
11,-769.292359,771.034400
12,-615.043592,614.640962
13,-834.818253,834.252569
14,-442.409839,444.161161


In [12]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-447.185842,449.057034
6,-771.663686,773.419270
7,-790.394996,788.423502
8,-833.574060,834.278275
9,-497.125198,495.760636
10,-217.985312,216.875478
11,-769.292359,771.034400
12,-615.043592,614.640962
13,-834.818253,834.252569
14,-442.409839,444.161161


In [13]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-447.18584173,  449.05703384],
       [-771.66368583,  773.41927008],
       [-790.39499576,  788.42350217],
       [-833.57405973,  834.27827513],
       [-497.12519799,  495.76063564],
       [-217.98531173,  216.87547791],
       [-769.29235871,  771.03440046],
       [-615.04359207,  614.64096231],
       [-834.81825253,  834.25256903],
       [-442.40983941,  444.16116136]])

In [14]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,892.371683,-898.114068
6,1541.327372,-1546.838540
7,1578.789992,-1576.847004
8,1665.148119,-1668.556550
9,992.250396,-991.521271
10,433.970623,-433.750956
11,1536.584717,-1542.068801
12,1228.087184,-1229.281925
13,1667.636505,-1668.505138
14,882.819679,-888.322323


In [15]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


In [16]:
# normalize inputs to unit domain [0,1]
normed_data = vocs.normalize_inputs(data)
normed_data

,a,b
5,0.448121,0.032202
6,0.772541,0.061104
7,0.789409,0.992873
8,0.833926,0.323946
9,0.496443,0.841141
10,0.217430,0.777458
11,0.770163,0.064490
12,0.614842,0.600657
13,0.834535,0.641421
14,0.443286,0.062170


In [17]:
# and denormalize
vocs.denormalize_inputs(normed_data)

,a,b
5,448.121438,-0.935596
6,772.541478,-0.877792
7,789.409249,0.985747
8,833.926167,-0.352108
9,496.442917,0.682281
10,217.430395,0.554917
11,770.163380,-0.871021
12,614.842277,0.201315
13,834.535411,0.282842
14,443.285500,-0.875661


# Error handling

In [18]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [19]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [20]:
data['a']  = np.nan

In [21]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [22]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [23]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
